In [2]:
# from sentence_transformers import SentenceTransformer, models
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from transformers import AutoTokenizer, AutoModel
import torch
import json 
from tqdm import tqdm
import numpy as np

# #Load the model
# base = models.Transformer('qna')
# pooling = models.Pooling(1024, pooling_mode='cls')
# model = SentenceTransformer(modules=[base, pooling])
model = AutoModel.from_pretrained('BAAI/bge-m3').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')


from langchain_core.embeddings import Embeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.documents import BaseDocumentCompressor
# from sentence_transformers.cross_encoder import CrossEncoder
from langchain.embeddings import OpenAIEmbeddings
from copy import deepcopy
import numpy as np
from langchain_core.pydantic_v1 import Extra, root_validator

def top_k(array, k):
    indices = np.argsort(array)[::-1][:k]  # Sort the indices in descending order and take top k
    return array[indices], indices



class AIEmbeddings(Embeddings):

    def embed_query(self, text):
        encoded = tokenizer(
            [text],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
        )
        encoded = {k: v.to('cuda') for k, v in encoded.items()}
        with torch.no_grad():
            embed = model(**encoded, return_dict=True).last_hidden_state[:, 0]
            embed = torch.nn.functional.normalize(embed, dim=-1)
        return embed[0].tolist()

    def embed_documents(self, texts):
        all_embeds = []
        docs = list(texts)
        for doc in docs:
            encoded = tokenizer(
                doc,
                padding=True,
                truncation=True,
                max_length=2048,
                return_tensors="pt",
            )
            print(len(encoded['input_ids'][0]))
            encoded = {k: v.to('cuda') for k, v in encoded.items()}
            with torch.no_grad():
                embed = model(**encoded, return_dict=True).last_hidden_state[:, 0]
                embed = torch.nn.functional.normalize(embed, dim=-1)
            all_embeds.append(embed[0].tolist())
        return all_embeds
    
import pandas as pd
from langchain.schema import Document #Format documents


sheet_list = ["Tài liệu - Quy định vận chuyển", "Tài liệu -  Quy định chung", "Tài liệu - Chính sách giá", "Tài liệu - Chính sách bồi hoàn", "Tài liệu - Ví"]
documents = list()
for sheet in sheet_list:
  df = pd.read_excel("data/Q&A.xlsx", sheet_name=sheet, skiprows=[0])
  text_1 = sheet
  text_2 = list(df['Mục'])
  text_3 = list(df['Nội dung'])
  titles = [f'{text_1}\n{text_2[i]}' for i in range(len(df))]
  contents = [f'{text_1}\n{text_2[i]}\n{text_3[i]}' for i in range(len(df))]
  for title, content in zip(titles, contents):
    #   print(content)
    #   print('-'*100)
      document = Document(
          page_content=content.strip(),
          metadata={
              'title': title
          }
      )
      documents.append(document)


class core_chat():
    def __init__(self, documents, embeddings):
        self.docs = documents
        self.embeddings = embeddings
        self.k = 3
        

    def setup_retriever(self):


        vectorstore = FAISS.from_documents(self.docs, self.embeddings)
        faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": self.k})

        self.ensemble_retriever = faiss_retriever
        return self.ensemble_retriever

    def get_relevant(self, question):
        docs_query = self.ensemble_retriever.get_relevant_documents(question)

        return docs_query
    
embeddings = AIEmbeddings()

core = core_chat(documents, embeddings)
retriever = core.setup_retriever()

onlyquestion = pd.read_csv("data/Clean_data_small.csv")

count = 0
onlyquestion["check"] = 0
for i in tqdm(onlyquestion.index):
    retrieved_docs = core.get_relevant(onlyquestion["question"][i])
    ret_title = [doc.metadata["title"] for doc in retrieved_docs]
    if onlyquestion["title"][i] in ret_title[0:3]:
        onlyquestion.loc[i,"check"] = 1

print(onlyquestion["check"].value_counts())




c:\Users\duyma\miniconda3\envs\CVenv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\duyma\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 1.72 GiB is allocated by PyTorch, and 4.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)